In [ ]:
import pyspark.sql.functions as f
from pyspark.sql import Window
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.getOrCreate()
w = Window().orderBy(f.lit('A'))

In [ ]:
#Leitura de CSV
df0 = spark.read.csv('work/HistoricoPedidos.csv',sep=';',header='false');

In [ ]:
  df1 = df0.select(df0._c0.cast("string").alias('CLIENTE'),\
                          df0._c1.cast("string").alias('TIPO'),\
                          f.substring(df0._c2,9,2).cast("int").alias('DIA_ABE'),\
                          f.substring(df0._c2,6,2).cast("int").alias('MES_ABE'),\
                          f.substring(df0._c2,1,4).cast("int").alias('ANO_ABE'),\
                          f.substring(df0._c2,12,2).cast("int").alias('HH_ABE'),\
                          f.substring(df0._c2,15,2).cast("int").alias('MM_ABE'),\
                          f.substring(df0._c3,9,2).cast("int").alias('DIA_FIM'),\
                          f.substring(df0._c3,6,2).cast("int").alias('MES_FIM'),\
                          f.substring(df0._c3,1,4).cast("int").alias('ANO_FIM'),\
                          f.substring(df0._c3,12,2).cast("int").alias('HH_FIM'),\
                          f.substring(df0._c3,15,2).cast("int").alias('MM_FIM'),\
                          df0._c4.cast("string").alias('FORMAPG'),\
                          df0._c5.cast("integer").alias('ENTREGA'),\
                          df0._c6.cast("string").alias('STATUS'),\
                          f.regexp_replace(f.regexp_replace(f.regexp_replace(f.trim(df0._c5),'[R$ ]',''),'[.]',''),',','.').alias('VALORFINAL'),\
                          df0._c8.cast("string").alias('ABRIR'),\
                          df0._c9.cast("string").alias('OBSPEDIDO'),\
                          df0._c10.cast("string").alias('OSBDELIVERY'))\
                          .withColumn('DATAINGESTAO', f.lit('20220620')) 
print(f'Historico Pedidos: {df1.count()}')

In [ ]:
# df1.filter(df1.DESCRICAO == "FRANQUIA").show()

In [ ]:
df2 = df1.distinct()

In [ ]:
# print(f'Contas a Pagar Formatados : {df2.count()}')
# df2.show()

In [ ]:
df2 = df1.dropna(how='all', subset='TIPO')
print(f'Historico Pedidos : {df2.count()}')

In [ ]:
# df_contasapagar = df3.select(f.row_number().over(w).alias('ID'),'*')

In [ ]:
# df_contasapagar.show(truncate=False)

In [ ]:
df2.write.format("parquet").mode("overwrite").parquet('work/historicopedidos')

In [ ]:
# dfMelhorDia = df_contasapagar.groupBy("MES_PG","CATEGORIA").agg(f.format_number(f.sum("VALORPG"),2).alias("total"),\
#                              f.format_number(f.avg("VALORPG"),2).alias("media"),\
#                              f.format_number(f.min("VALORPG"),2).alias("menor"),\
#                              f.format_number(f.max("VALORPG"),2).alias("maior")).\
#     orderBy(["MES_PG","total"],ascending=[0, 0])

In [ ]:
dfMelhorDia = df2.groupBy("MES_ABE","FORMAPG").agg(f.round(f.sum("VLLIQUIDO"),2).alias("total"),\
                             f.count("MES_HIS").alias("Compras"),\
                             f.round(f.avg("VLLIQUIDO"),2).alias("media"),\
                             f.round(f.min("VLLIQUIDO"),2).alias("menor"),\
                             f.round(f.max("VLLIQUIDO"),2).alias("maior")).\
    orderBy(["MES_HIS",'FORMAPG'],ascending=[1, 1])

In [ ]:
dfMelhorDia.show(truncate=False,n=1000)
